In [4]:
import json
import requests
import time
from typing import List, Dict

# Load API key from environment or config
CRYPTO_PANIC_API_KEY = "c61180e0b531b0c266add9888bf138dca4712261"
BASE_URL = "https://cryptopanic.com/api/v1/posts/"

def load_symbol_map(path: str) -> Dict[str, str]:
    with open("/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/assets_input.json", "r", encoding="utf-8") as f:
        return json.load(f)

def fetch_news_for_assets(
    asset_names: List[str],
    symbol_map: Dict[str, str],
    max_per_asset: int = 5
) -> Dict[str, List[Dict[str, str]]]:
    results = {}
    for asset in asset_names:
        symbol = symbol_map.get(asset)
        if not symbol:
            print(f"Skipping unknown asset: {asset}")
            continue

        params = {
            "auth_token": CRYPTO_PANIC_API_KEY,
            "currencies": symbol,
            "filter": "hot"
        }

        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            posts = response.json().get("results", [])[:max_per_asset]
            results[asset] = [{
                "text": post["title"],
                "source": post["url"]
            } for post in posts]
        else:
            print(f"Error fetching {asset}: HTTP {response.status_code}")
        time.sleep(1)  # Avoid hitting rate limits

    return results

def build_asset_json(news_dict: Dict[str, List[Dict[str, str]]]) -> List[Dict[str, object]]:
    return [
        {
            "asset_name": asset,
            "texts": entries
        }
        for asset, entries in news_dict.items()
    ]

def save_to_json(data: List[Dict[str, object]], output_path: str):
    with open("/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/assets_output.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

# Example usage
if __name__ == "__main__":
    symbol_map_path = "sentiment_analysis/config/asset_symbol_map.json"
    output_path = "sentiment_analysis/data/input_assets.json"
    asset_names = ["LINK", "ETH", "USDT", "BTC", "SOL", "ARB", "MKR", "AVAX", "OP", "AAVE"]

    symbol_map = load_symbol_map(symbol_map_path)
    news = fetch_news_for_assets(asset_names, symbol_map)
    asset_json = build_asset_json(news)
    save_to_json(asset_json, output_path)

    print(f"✅ Asset news saved to {output_path}")


✅ Asset news saved to sentiment_analysis/data/input_assets.json


In [1]:
import json
import requests
import time
from datetime import datetime, timedelta
from typing import List, Dict, Tuple

# === Configuration ===
CRYPTO_PANIC_API_KEY = "c61180e0b531b0c266add9888bf138dca4712261"
BASE_URL = "https://cryptopanic.com/api/v1/posts/"
MAX_PER_ASSET = 5
DAYS_BACK = 7

# === File Paths ===
INPUT_ASSETS_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/assets_input.json"
SYMBOL_MAP_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/config/asset_symbol_map.json"
OUTPUT_ASSETS_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/assets_output.json"
SKIPPED_ASSETS_PATH = "/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/skipped_assets.json"

# === Helpers ===

def load_asset_names(path: str) -> List[str]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_symbol_map(path: str) -> Dict[str, str]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def get_recent_articles(symbol: str, filter_type: str) -> List[Dict[str, str]]:
    params = {
        "auth_token": CRYPTO_PANIC_API_KEY,
        "currencies": symbol,
        "filter": filter_type
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        print(f"[{symbol}] Error {response.status_code} with filter={filter_type}")
        return []

    cutoff = datetime.utcnow() - timedelta(days=DAYS_BACK)
    results = []
    for post in response.json().get("results", []):
        try:
            pub_time = datetime.strptime(post["published_at"], "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
        except Exception:
            continue
        if pub_time >= cutoff:
            results.append({
                "text": post["title"],
                "source": post["url"],
                "published_at": pub_time.isoformat()
            })
        if len(results) >= MAX_PER_ASSET:
            break
    return results

def fetch_filtered_news(asset_names: List[str], symbol_map: Dict[str, str]) -> Tuple[List[Dict], List[Dict]]:
    included_assets = []
    skipped_assets = []
    cutoff = (datetime.utcnow() - timedelta(days=DAYS_BACK)).isoformat()

    for asset in asset_names:
        symbol = symbol_map.get(asset)
        if not symbol:
            skipped_assets.append({
                "asset_name": asset,
                "status": "unknown_symbol",
                "risk_level": None,
                "reason": "Asset symbol not found in symbol map",
                "priority": None
            })
            continue

        articles = get_recent_articles(symbol, filter_type="hot")

        if not articles:
            print(f"[{asset}] Fallback to filter=new")
            articles = get_recent_articles(symbol, filter_type="new")

        if articles:
            included_assets.append({
                "asset_name": asset,
                "texts": articles
            })
        else:
            skipped_assets.append({
                "asset_name": asset,
                "status": "no_recent_data",
                "risk_level": None,
                "reason": f"No qualifying articles found after {cutoff}",
                "priority": None
            })

        time.sleep(1)  # Avoid rate limits

    return included_assets, skipped_assets

def save_json(data: object, path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

# === Main Execution ===
if __name__ == "__main__":
    asset_names = load_asset_names(INPUT_ASSETS_PATH)
    symbol_map = load_symbol_map(SYMBOL_MAP_PATH)

    included, skipped = fetch_filtered_news(asset_names, symbol_map)

    save_json(included, OUTPUT_ASSETS_PATH)
    save_json(skipped, SKIPPED_ASSETS_PATH)

    print(f"✅ Recent news saved to: {OUTPUT_ASSETS_PATH}")
    print(f"⚠️ Skipped assets saved to: {SKIPPED_ASSETS_PATH}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/config/asset_symbol_map.json'